In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = (
    pl.DataFrame(
        {
            "f": np.sin(list(range(size))),
            "time_idx": range(size),
            "dummy": ["a"] * (size // 2) + ["b"] * (size // 2),
            "actual": np.round(np.random.random(size=size)).astype(np.int32),
            "predicted": np.random.random(size=size),
            "dummy_groups": ["a"] * (size // 2) + ["b"] * (size // 2),
        }
    )
    .with_columns(
        pds.random(0.0, 1.0).alias("x1"),
        pds.random(0.0, 1.0).alias("x2"),
        pds.random(0.0, 1.0).alias("x3"),
        pds.random(0.0, 1.0).alias("a"),
        pds.random(0.0, 1.0).alias("b"),
    )
    .with_columns(
        y=pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
        y2=pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001,
    )
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.604321,"""a""",0.15237,0.089051,0.557941,0.957239,0.697476,-0.78732,0.00418
0.841471,1,"""a""",0,0.587967,"""a""",0.005057,0.994689,0.135608,0.521284,0.437935,0.095838,0.434758
0.909297,2,"""a""",0,0.349146,"""a""",0.011923,0.385763,0.231373,0.161492,0.254986,-0.229514,0.152057
0.14112,3,"""a""",1,0.935919,"""a""",0.642712,0.757879,0.991374,0.177408,0.830842,-1.163207,0.325529
-0.756802,4,"""a""",1,0.334662,"""a""",0.878021,0.027712,0.357115,0.532788,0.750502,-0.395648,0.090906


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(pds.jaccard_col("x1", pl.col("x2")))

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant
# compact representation of fft output.
df.select(pds.rfft("f")).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(pds.rfft("f", return_full=True)).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve(
        "f", [-1, 0, 0, 0, 1], mode="full", method="fft"
    ),  # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode="full", method="direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode="full", method="direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.957239,-0.697476
-0.841471,-0.521284,-0.437935
-0.909297,-0.161492,-0.254986
-0.14112,-0.177408,-0.830842
0.756802,0.42445,-0.053025


In [7]:
# Linear Regression
df.select(pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False))

coeffs
list[f64]
"[-0.491007, -0.33619]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=[pl.col("y"), pl.col("y2")], add_bias=False)
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.491007, -0.33619]","[0.087306, 0.407626]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=[pl.col("y"), pl.col("y2")], add_bias=False)
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.491007, -0.336191]","[0.087306, 0.407626]"


In [10]:
pds.Config.LIN_REG_EXPR_F64 = True  # pds.Config or pds.config will both work

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries,
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)",
        "exp(x2)",
        "sin(x3)",
        target="y",
        add_bias=True,
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.220658,0.001681,131.270637,0.0,0.217363,0.223953,0.994483,0.994481
"""exp(x2)""",0.175519,0.000673,260.903142,0.0,0.1742,0.176837,0.994483,0.994481
"""sin(x3)""",-1.744938,0.001337,-1305.032432,0.0,-1.747559,-1.742317,0.994483,0.994481
"""__bias__""",-0.110001,0.001511,-72.789375,0.0,-0.112963,-0.107038,0.994483,0.994481


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries,
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)",
        "exp(x2)",
        "sin(x3)",
        target="y",
        add_bias=True,
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.220656,0.001681,131.269455,0.0,0.217361,0.223951,0.994483,0.994481
"""exp(x2)""",0.175519,0.000673,260.903778,0.0,0.174201,0.176838,0.994483,0.994481
"""sin(x3)""",-1.744941,0.001337,-1305.034546,0.0,-1.747562,-1.74232,0.994483,0.994481
"""__bias__""",-0.11,0.001511,-72.788811,0.0,-0.112962,-0.107038,0.994483,0.994481


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False).over(
        pl.col("dummy")
    ),
)

dummy,coeffs
str,list[f64]
"""a""","[-0.497065, -0.339606]"
"""a""","[-0.497065, -0.339606]"
"""a""","[-0.497065, -0.339606]"
"""a""","[-0.497065, -0.339606]"
"""a""","[-0.497065, -0.339606]"
…,…
"""b""","[-0.484646, -0.333059]"
"""b""","[-0.484646, -0.333059]"
"""b""","[-0.484646, -0.333059]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg("x1", pl.col("x2"), target="y", add_bias=False, return_pred=True).alias(
        "prediction"
    ),
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.15237,0.089051,-0.78732,-0.104753,-0.682568
0.005057,0.994689,0.095838,-0.336888,0.432726
0.011923,0.385763,-0.229514,-0.135544,-0.09397
0.642712,0.757879,-1.163207,-0.570368,-0.592839
0.878021,0.027712,-0.395648,-0.440431,0.044783


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), add_bias=False)
)

dummy,coeffs
str,list[f64]
"""b""","[-0.484646, -0.333059]"
"""a""","[-0.497065, -0.339606]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(pl.col("x1"), pl.col("x2"), target=pl.col("y"), l1_reg=0.1, add_bias=False)
)

dummy,coeffs
str,list[f64]
"""b""","[-0.31023, -0.168204]"
"""a""","[-0.332789, -0.164381]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual=pl.col("y"),
        pred=pds.lin_reg(
            pl.col("x1"),
            pl.col("x2"),
            target=pl.col("y"),
            l1_reg=0.1,
            return_pred=True,
            add_bias=False,
        ).struct.field("pred"),
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.539024
"""b""",-0.544072


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg("x1", "x2", target="y", window_size=5, null_policy="zero").alias("result"),
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.78732,0.15237,0.089051,null,null
0.095838,0.005057,0.994689,null,null
-0.229514,0.011923,0.385763,null,null
-1.163207,0.642712,0.757879,null,null
-0.395648,0.878021,0.027712,"[-0.883798, -0.280806]",-0.783775
…,…,…,…,…
-0.37273,0.339425,0.289881,"[0.145721, -1.218511]",-0.303762
-1.083488,0.10923,0.111354,"[0.503313, -1.295788]",-0.089315
0.00281,0.084865,0.50878,"[0.276498, -0.908577]",-0.4388


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(pds.query_cond_entropy("y", "x1"))

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(pds.singular_values("a", "b", "x1"))

a
list[f64]
"[28.970883, 28.779817, 28.703428]"


In [22]:
# Singular values + The principal components
df.select(pds.pca("a", "b")).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.818884,"[0.744481, 0.667643]"
28.714139,"[-0.667643, 0.744481]"


In [23]:
# PC1
df.select(pds.principal_components("a", "b", k=1).alias("principal_components")).unnest(
    "principal_components"
).head()

pc1
f64
0.471102
-0.026739
-0.416742
-0.020427
0.190509


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo"),
).unnest("combo")

dummy_groups,l2,log loss,precision,recall,f,avg_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.333009,0.993546,0.502002,0.503614,0.502807,0.504389,0.502175
"""a""",0.333389,0.99792,0.501794,0.498416,0.500099,0.507728,0.499357


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame(
    {"sen": ["Hello, world! I'm going to church."] * size, "word": ["words", "word"] * (size // 2)}
)
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
df2.select(pds.str_leven("word", pl.lit("world"))).head()

word
u32
2
1
2
1
2


In [27]:
# Damerau-Levenshtein
df2.select(pds.str_d_leven("word", pl.lit("world"))).head()

word
u32
2
1
2
1
2


In [28]:
df2.select(  # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim=True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [29]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1)  # <= 1.
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [30]:
df = pl.DataFrame(
    {
        "word": ["apple", "banana", "pineapple", "asasasas", "sasasass"],
        "other_data": [1, 2, 3, 4, 5],
    }
)
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [31]:
df.select(
    # Nearest string
    pds.str_nearest("word", word="banana")
)

word
str
"""banana"""


In [32]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab=gibberish,
        threshold=0.5,
        metric="lv",  # Levenshtein similarity. Other options: dleven, osa, jw
        strategy="any",  # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [33]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias(
        "Optimal String Alignment - apples"
    ),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)

asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [34]:
import numpy as np

df = pl.DataFrame({"a": [None, None] + list(np.random.normal(size=998))})
df.head()

a
f64
null
null
-1.301585
-1.833191
-0.367643


In [35]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean=0.5, std=1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_normal(mean=0.5, std=1.0).alias("random_normal"))
    .alias("random_normal_that_respects_null_of_a"),
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,0.343075,null
null,-0.030428,null
-1.301585,0.248074,-1.39454
-1.833191,-0.348191,0.771747
-0.367643,-0.241624,0.901401


In [36]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size=1, max_size=5).alias("random_str"),
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_str(min_size=1, max_size=5))
    .alias("random_str_that_respects_null_of_a"),
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Jf5L""",null
null,"""qHT""",null
-1.301585,"""f""","""oHj"""
-1.833191,"""UbwY""","""Odx"""
-0.367643,"""eEjC""","""Vo"""


In [37]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random_str(min_size=5, max_size=5))
    .alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-1.301585,"""tyVVm"""
-1.833191,"""cjqOE"""
-0.367643,"""l8fd0"""


In [38]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1"),
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    pl.when(pl.col("a").is_null())
    .then(None)
    .otherwise(pds.random(lower=0.0, upper=pl.col("a").max()).alias("test2")),
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.402744,null,-0.403
null,-0.071263,null,-0.07107
-1.301585,0.214382,1.418486,0.214495
-1.833191,1.063134,1.683947,1.063205
-0.367643,0.036393,0.320071,0.036246


In [39]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.313596,2.055479,7
null,0.706754,-0.370513,5
-1.301585,0.825359,1.354362,7
-1.833191,0.675932,-0.968769,8
-0.367643,0.11364,0.857566,4


In [40]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test"),
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics"),
    pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue"),
    pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics"),
    pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue"),
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-1.413054,0.157856,3.59819,0.165449


In [41]:
size = 5_000
df = pl.DataFrame(
    {
        "market_id": range(size),
    }
).with_columns(
    pl.col("market_id").mod(3),
    var1=pds.random(),
    var2=pds.random(),
    category_1=pds.random_int(0, 5),
    category_2=pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.957087,0.398313,3,7
1,0.878786,0.719574,1,7
2,0.577419,0.810361,3,8
0,0.623346,0.432069,3,2
1,0.25289,0.377306,4,4


In [42]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group="category_1").alias("f-test"),
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-1.133786,0.256911}","{38.169559,0.371075}","{1.671814,0.153553}"


In [43]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group="category_1").alias("f-test"),
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-1.742671,0.081483} ┆ {43.903429,0.171519} ┆ {1.058412,0.375669} │
│ 1         ┆ {0.329208,0.742019}  ┆ {38.421037,0.360394} ┆ {1.518306,0.194288} │
│ 2         ┆ {-0.529579,0.596439} ┆ {24.519703,0.92653}  ┆ {0.883226,0.473139} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [44]:
# Benford's law
df.select(first_digit_cnt=pds.query_first_digit_cnt(pl.col("var1")).explode()).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution=pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
553,0.1106
568,0.1136
573,0.1146
583,0.1166
598,0.1196
562,0.1124
522,0.1044
553,0.1106
488,0.0976


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [45]:
import polars_ds as pds

size = 2000
df = pl.DataFrame(
    {
        "id": range(size),
    }
).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32),
)

In [46]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"),
        "var2",
        "var3",  # Columns used as the coordinates in n-d space, str | pl.Expr
        r=0.1,  # radius
        dist="inf",  # L Infinity distance
        parallel=True,
    ).alias("nb_l_inf_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.157738,0.831088,0.363077,0.922942,6.513781,17
1,0.429605,0.666567,0.812754,0.734055,0.321603,24
2,0.513745,0.462735,0.437161,0.873912,1.765905,17
3,0.305626,0.838603,0.124756,0.626971,0.812412,14
4,0.33908,0.989033,0.936281,0.650139,1.897958,12


In [47]:
df.with_columns(
    pds.query_nb_cnt(
        "var1",
        "var2",
        "var3",  # Columns used as the coordinates in n-d space, str | pl.Expr
        r=pl.col("r"),  # radius be an expression too
        dist="l1",  # L 1 distance
        parallel=True,
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.157738,0.831088,0.363077,0.922942,6.513781,885
1,0.429605,0.666567,0.812754,0.734055,0.321603,743
2,0.513745,0.462735,0.437161,0.873912,1.765905,1353
3,0.305626,0.838603,0.124756,0.626971,0.812412,347
4,0.33908,0.989033,0.936281,0.650139,1.897958,233


In [48]:
# Get ids of the k nearest neighbors.
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        index="id",  # pl.col("id"), str | pl.Expr
        k=3,
        dist="l2",  # squared l2
        parallel=True,
    ).alias("best friends")
).head()

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.157738,0.831088,0.363077,0.922942,6.513781,"[0, 612, … 282]"
1,0.429605,0.666567,0.812754,0.734055,0.321603,"[1, 224, … 549]"
2,0.513745,0.462735,0.437161,0.873912,1.765905,"[2, 554, … 1613]"
3,0.305626,0.838603,0.124756,0.626971,0.812412,"[3, 589, … 1977]"
4,0.33908,0.989033,0.936281,0.650139,1.897958,"[4, 1145, … 275]"


In [49]:
# Get all neighbors within radius r, call them best friends
print(
    df.select(
        pl.col("id"),
        pds.query_radius_ptwise(
            pl.col("var1"),
            pl.col("var2"),
            pl.col("var3"),  # Columns used as the coordinates in 3d space
            index=pl.col("id"),
            r=0.1,
            dist="l2",  # actually this is squared l2
            parallel=True,
        ).alias("best friends"),
    )
    .with_columns(  # -1 to remove the point itself
        (pl.col("best friends").list.len() - 1).alias("best friends count")
    )
    .head()
)

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 612, … 1249] ┆ 11                 │
│ 1   ┆ [1, 224, … 1867] ┆ 8                  │
│ 2   ┆ [2, 554, … 815]  ┆ 8                  │
│ 3   ┆ [3, 589, … 266]  ┆ 7                  │
│ 4   ┆ [4, 1145, … 127] ┆ 6                  │
└─────┴──────────────────┴────────────────────┘


In [50]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        index=pl.col("id"),
        k=3,
        dist="l2",  # actually this is squared l2
        parallel=True,
        return_dist=True,
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.157738,0.831088,0.363077,0.922942,6.513781,"[0, 612, … 282]","[0.0, 0.051489, … 0.063251]"
1,0.429605,0.666567,0.812754,0.734055,0.321603,"[1, 224, … 549]","[0.0, 0.034903, … 0.046191]"
2,0.513745,0.462735,0.437161,0.873912,1.765905,"[2, 554, … 1613]","[0.0, 0.053701, … 0.067293]"
3,0.305626,0.838603,0.124756,0.626971,0.812412,"[3, 589, … 1977]","[0.0, 0.022789, … 0.071176]"
4,0.33908,0.989033,0.936281,0.650139,1.897958,"[4, 1145, … 275]","[0.0, 0.044816, … 0.061075]"


In [51]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),
        pl.col("var3"),  # Columns used as the coordinates in n-d space
        pt=[0.5, 0.5, 0.5],
        r=0.2,
        dist="l2",  # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
1,0.429605,0.666567,0.812754,0.734055,0.321603
2,0.513745,0.462735,0.437161,0.873912,1.765905
8,0.484307,0.92643,0.538635,0.982432,1.207466
9,0.549613,0.399481,0.46647,0.201114,1.095284
11,0.603748,0.275982,0.205979,0.110363,8.094406


In [52]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),  # Columns used as the coordinates in n-d space
        pt=[0.5, 0.5],
        r=10,  # in km
        dist="h",
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
2,0.513745,0.462735,0.437161,0.873912,1.765905
70,0.542059,0.50288,0.52487,0.15167,2.539043
87,0.470472,0.512549,0.349451,0.402647,2.285905
100,0.472096,0.515557,0.746271,0.835733,3.043255
251,0.478547,0.541688,0.946517,0.125863,6.048753


In [53]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"),
        pl.col("var2"),
        pt=[0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r=pl.col("rh"),
        dist="h",
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
251,0.478547,0.541688,0.946517,0.125863,6.048753
536,0.565317,0.507856,0.068942,0.23305,8.484623
554,0.50381,0.455882,0.384834,0.566188,7.919236
873,0.539223,0.533934,0.136974,0.053008,7.776476
1035,0.478181,0.526502,0.623084,0.183265,5.055358


In [54]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"),
        pl.col("var2"),
        index=pl.col("id"),
        r=0.02,
        dist="l2",
    ).alias("friends"),
).with_columns(pl.col("friends").list.len().alias("count"))
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1843, 1316]",3
1,"[1, 1372, … 1132]",4
2,"[2, 554, 1475]",3
3,"[3, 141, … 597]",5
4,"[4, 1374]",2


# Compatibility

## Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures. 

## Using PDS with Narwhals

Limited

In [55]:
import pandas as pd
import numpy as np
import polars as pl
import polars_ds as pds
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_13194/181413171.py:5: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.142662,0,0,"""4h""","""P"""
0.0,0.22716,0,9,"""Yu""","""Fw"""
1.0,0.134794,0,0,"""cQ""","""L7"""
0.0,0.815109,0,9,"""EY""","""sm"""
0.0,0.074559,1,4,"""V""","""mG"""


In [56]:
df_pd = df.to_pandas()

In [57]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

0    0.3
Name: , dtype: float64

In [58]:
# Polars Series
print(pds2.query_roc_auc(df["actual"], df["predicted"]))
# NumPy
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_numpy(), df["predicted"].to_numpy()))
pds2.return_numpy = False
# Pandas
print(pds2.query_roc_auc(df["actual"].to_pandas(), df["predicted"].to_pandas()))
# PyArrow
# Arrow series can be inputs, but the output cannot be converted correctly. Please let me know if you have a fix.
# The work around is to use NumPy for Arrow
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_arrow(), df["predicted"].to_arrow()))
# Other array-protocal compatible inputs
# print(pds2.query_roc_auc(df["actual"].to_jax(), df["predicted"].to_jax()))

pds2.return_numpy = False

shape: (1,)
Series: '' [f64]
[
	0.502437
]
[0.50243727]
0    0.502437
Name: , dtype: float64
[0.50243727]


In [59]:
# NumPy Arrays
pds2.psi(
    np.random.random(size=1000),
    np.random.random(size=1000),
    n_bins=5,
)

array([0.02079848])

In [60]:
df_pd = df.to_pandas()
df_pd["levenshtein_dist"] = pds2.str_leven(df_pd["s1"], df_pd["s2"])
df_pd.head()

,actual,predicted,0-2,0-9,s1,s2,levenshtein_dist
0,1.0,0.142662,0,0,4h,P,2
1,0.0,0.227160,0,9,Yu,Fw,2
2,1.0,0.134794,0,0,cQ,L7,2
3,0.0,0.815109,0,9,EY,sm,2
4,0.0,0.074559,1,4,V,mG,2


In [61]:
# If you are using Narwhals, well, Narwhal expressions are not Polars expressions.
# Using the pds2 module, you can run pds functions in map_batches, but this is limited to 1 input column.

import narwhals as nw

df_nw = nw.from_native(df_pd)
df_nw.with_columns(
    nw_levenshtein_dist=nw.col("s1").map_batches(
        lambda s: pds2.str_leven(s.to_numpy(), pl.lit("k9"))
    )
).head()

┌─────────────────────────────────────────────────────────────────────────────┐
|                             Narwhals DataFrame                              |
|-----------------------------------------------------------------------------|
|   actual  predicted  0-2  0-9  s1  s2  levenshtein_dist  nw_levenshtein_dist|
|0     1.0   0.142662    0    0  4h   P                 2                    2|
|1     0.0   0.227160    0    9  Yu  Fw                 2                    2|
|2     1.0   0.134794    0    0  cQ  L7                 2                    2|
|3     0.0   0.815109    0    9  EY  sm                 2                    2|
|4     0.0   0.074559    1    4   V  mG                 2                    2|
└─────────────────────────────────────────────────────────────────────────────┘